In [1]:
# Packages / libraries
import os #provides functions for interacting with the operating system
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

# To install sklearn type "pip install numpy scipy scikit-learn" to the anaconda terminal

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(12,10)})

# import sys
# !conda list Check the packages installed

In [2]:
# Loading the cumulative raw data

raw_data_confirmed = pd.read_csv('analysis/data/africa_cases.csv')
raw_data_deaths = pd.read_csv('analysis/data/africa_deaths.csv')
raw_data_Recovered = pd.read_csv('analysis/data/africa_recover.csv')

print("The Shape of Confirmed is: ", raw_data_confirmed.shape)
print("The Shape of Confirmed is: ", raw_data_deaths.shape)
print("The Shape of Confirmed is: ", raw_data_Recovered.shape)

raw_data_confirmed.head()

The Shape of Confirmed is:  (55, 129)
The Shape of Confirmed is:  (55, 129)
The Shape of Confirmed is:  (55, 129)


Province/State Country/Region      Lat     Long  Region        Subregion  \
0             NaN        Algeria  28.0339   1.6596  Africa  Northern Africa   
1             NaN         Angola -11.2027  17.8739  Africa    Middle Africa   
2             NaN          Benin   9.3077   2.3158  Africa   Western Africa   
3             NaN   Burkina Faso  12.2383  -1.5616  Africa   Western Africa   
4             NaN     Cabo Verde  16.5388 -23.0418  Africa   Western Africa   

   1/22/20  1/23/20  1/24/20  1/25/20  ...  5/14/20  5/15/20  5/16/20  \
0        0        0        0        0  ...     6442     6629     6821   
1        0        0        0        0  ...       48       48       48   
2        0        0        0        0  ...      339      339      339   
3        0        0        0        0  ...      773      780      782   
4        0        0        0        0  ...      315      326      328   

   5/17/20  5/18/20  5/19/20  5/20/20  5/21/20  5/22/20  5/23/20  
0     7019     7201     7377     7542     7728     7918     8113  
1       48       50       52       52       58       60       61  
2      339      339      130      130      135      135      135  
3      796      796      796      809      812      814      814  
4      328      328      335      349      356      362      371  

[5 rows x 129 columns]

In [3]:
# Un-Pivoting the data

raw_data_confirmed2 = pd.melt(raw_data_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long','Region','Subregion'], var_name=['Date'])
raw_data_deaths2 = pd.melt(raw_data_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long','Region','Subregion'], var_name=['Date'])
raw_data_Recovered2 = pd.melt(raw_data_Recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long','Region','Subregion'], var_name=['Date'])


print("The Shape of Cornfirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Cornfirmed is: ", raw_data_deaths2.shape)
print("The Shape of Cornfirmed is: ", raw_data_Recovered2.shape)


raw_data_confirmed2.head()

The Shape of Cornfirmed is:  (6765, 8)
The Shape of Cornfirmed is:  (6765, 8)
The Shape of Cornfirmed is:  (6765, 8)


Province/State Country/Region      Lat     Long  Region        Subregion  \
0             NaN        Algeria  28.0339   1.6596  Africa  Northern Africa   
1             NaN         Angola -11.2027  17.8739  Africa    Middle Africa   
2             NaN          Benin   9.3077   2.3158  Africa   Western Africa   
3             NaN   Burkina Faso  12.2383  -1.5616  Africa   Western Africa   
4             NaN     Cabo Verde  16.5388 -23.0418  Africa   Western Africa   

      Date  value  
0  1/22/20      0  
1  1/22/20      0  
2  1/22/20      0  
3  1/22/20      0  
4  1/22/20      0

In [4]:
raw_data_confirmed2['Date'] = pd.to_datetime(raw_data_confirmed2['Date'])
raw_data_deaths2['Date'] = pd.to_datetime(raw_data_deaths2['Date'])
raw_data_Recovered2['Date'] = pd.to_datetime(raw_data_Recovered2['Date'])

In [5]:
# Renaming the Values
raw_data_confirmed2.columns = raw_data_confirmed2.columns.str.replace('value', 'Confirmed')
raw_data_deaths2.columns = raw_data_deaths2.columns.str.replace('value', 'Deaths')
raw_data_Recovered2.columns = raw_data_Recovered2.columns.str.replace('value', 'Recovered')

In [6]:
# Investigating the NULL values
raw_data_confirmed2.isnull().sum()

Province/State    6765
Country/Region       0
Lat                  0
Long                 0
Region               0
Subregion            0
Date                 0
Confirmed            0
dtype: int64

In [7]:
# printing shapes before the join
print("The Shape of Cornirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Cornirmed is: ", raw_data_deaths2.shape)
print("The Shape of Cornirmed is: ", raw_data_Recovered2.shape)

The Shape of Cornirmed is:  (6765, 8)
The Shape of Cornirmed is:  (6765, 8)
The Shape of Cornirmed is:  (6765, 8)


In [8]:
# Full Joins

# Confirmed with Deaths
full_join = raw_data_confirmed2.merge(raw_data_deaths2[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'outer', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of first join: ", full_join.shape)

# full join with Recovered
full_join = full_join.merge(raw_data_Recovered2[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'outer', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of second join: ", full_join.shape)

full_join.head()

Shape of first join:  (6765, 9)
Shape of second join:  (6765, 10)


Province/State Country/Region      Lat     Long  Region        Subregion  \
0             NaN        Algeria  28.0339   1.6596  Africa  Northern Africa   
1             NaN         Angola -11.2027  17.8739  Africa    Middle Africa   
2             NaN          Benin   9.3077   2.3158  Africa   Western Africa   
3             NaN   Burkina Faso  12.2383  -1.5616  Africa   Western Africa   
4             NaN     Cabo Verde  16.5388 -23.0418  Africa   Western Africa   

        Date  Confirmed  Deaths  Recovered  
0 2020-01-22          0       0          0  
1 2020-01-22          0       0          0  
2 2020-01-22          0       0          0  
3 2020-01-22          0       0          0  
4 2020-01-22          0       0          0

In [9]:
full_join.isnull().sum()

Province/State    6765
Country/Region       0
Lat                  0
Long                 0
Region               0
Subregion            0
Date                 0
Confirmed            0
Deaths               0
Recovered            0
dtype: int64

In [10]:
# Adding Month and Year as a new Column
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

In [11]:
full_join.head()

Province/State Country/Region      Lat     Long  Region        Subregion  \
0             NaN        Algeria  28.0339   1.6596  Africa  Northern Africa   
1             NaN         Angola -11.2027  17.8739  Africa    Middle Africa   
2             NaN          Benin   9.3077   2.3158  Africa   Western Africa   
3             NaN   Burkina Faso  12.2383  -1.5616  Africa   Western Africa   
4             NaN     Cabo Verde  16.5388 -23.0418  Africa   Western Africa   

        Date  Confirmed  Deaths  Recovered Month-Year  
0 2020-01-22          0       0          0   Jan-2020  
1 2020-01-22          0       0          0   Jan-2020  
2 2020-01-22          0       0          0   Jan-2020  
3 2020-01-22          0       0          0   Jan-2020  
4 2020-01-22          0       0          0   Jan-2020

In [12]:
#############################################################################################
######################## Braking the numbers by Day #########################################
#############################################################################################

# filtering data to Anhui to give you an example

#creating a new df    
test = full_join[full_join['Country/Region'] == 'Senegal']

#creating a new df    
full_join2 = test.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = test.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'outer',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']


test.head()
full_join2.head()
full_join3.head()

Province/State Country/Region      Lat     Long  Region       Subregion  \
0             NaN        Senegal  14.4974 -14.4524  Africa  Western Africa   
1             NaN        Senegal  14.4974 -14.4524  Africa  Western Africa   
2             NaN        Senegal  14.4974 -14.4524  Africa  Western Africa   
3             NaN        Senegal  14.4974 -14.4524  Africa  Western Africa   
4             NaN        Senegal  14.4974 -14.4524  Africa  Western Africa   

        Date  Confirmed  Deaths  Recovered Month-Year  Confirmed - 1  \
0 2020-01-22        0.0     0.0        0.0   Jan-2020            NaN   
1 2020-01-23        0.0     0.0        0.0   Jan-2020            0.0   
2 2020-01-24        0.0     0.0        0.0   Jan-2020            0.0   
3 2020-01-25        0.0     0.0        0.0   Jan-2020            0.0   
4 2020-01-26        0.0     0.0        0.0   Jan-2020            0.0   

   Deaths - 1  Recovered - 1   Date - 1 Date Minus 1  Confirmed Daily  
0         NaN            NaN        NaT          NaT              NaN  
1         0.0            0.0 2020-01-23   2020-01-22              0.0  
2         0.0            0.0 2020-01-24   2020-01-23              0.0  
3         0.0            0.0 2020-01-25   2020-01-24              0.0  
4         0.0            0.0 2020-01-26   2020-01-25              0.0

In [13]:
#############################################################################################
######################## Braking the numbers by Day #########################################
#############################################################################################

## Applying it on all dataset

#creating a new df    
full_join2 = full_join.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

#minus_onedf.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1'}, inplace=True)

full_join3.head()

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']

print(full_join3.shape)

(6765, 19)


In [15]:
full_join3.head()

Province/State Country/Region      Lat     Long  Region        Subregion  \
0        Algeria        Algeria  28.0339   1.6596  Africa  Northern Africa   
1         Angola         Angola -11.2027  17.8739  Africa    Middle Africa   
2          Benin          Benin   9.3077   2.3158  Africa   Western Africa   
3   Burkina Faso   Burkina Faso  12.2383  -1.5616  Africa   Western Africa   
4     Cabo Verde     Cabo Verde  16.5388 -23.0418  Africa   Western Africa   

        Date  Confirmed  Deaths  Recovered Month-Year  Confirmed - 1  \
0 2020-01-22        0.0     0.0        0.0   Jan-2020            NaN   
1 2020-01-22        0.0     0.0        0.0   Jan-2020            NaN   
2 2020-01-22        0.0     0.0        0.0   Jan-2020            NaN   
3 2020-01-22        0.0     0.0        0.0   Jan-2020            NaN   
4 2020-01-22        0.0     0.0        0.0   Jan-2020            NaN   

   Deaths - 1  Recovered - 1 Date - 1 Date Minus 1  Confirmed Daily  \
0         NaN            NaN      NaT          NaT              NaN   
1         NaN            NaN      NaT          NaT              NaN   
2         NaN            NaN      NaT          NaT              NaN   
3         NaN            NaN      NaT          NaT              NaN   
4         NaN            NaN      NaT          NaT              NaN   

   Deaths Daily  Recovered Daily  
0           NaN              NaN  
1           NaN              NaN  
2           NaN              NaN  
3           NaN              NaN  
4           NaN              NaN

In [14]:
# Additing manually the numbers for first day

full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

# deleting columns
del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

/Users/datascientistenthusiast/.local/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
full_join3.head()

Province/State Country/Region      Lat     Long  Region        Subregion  \
0             NaN        Algeria  28.0339   1.6596  Africa  Northern Africa   
1             NaN         Angola -11.2027  17.8739  Africa    Middle Africa   
2             NaN          Benin   9.3077   2.3158  Africa   Western Africa   
3             NaN   Burkina Faso  12.2383  -1.5616  Africa   Western Africa   
4             NaN     Cabo Verde  16.5388 -23.0418  Africa   Western Africa   

        Date  Confirmed  Deaths  Recovered Month-Year  Confirmed Daily  \
0 2020-01-22          0       0          0   Jan-2020              0.0   
1 2020-01-22          0       0          0   Jan-2020              0.0   
2 2020-01-22          0       0          0   Jan-2020              0.0   
3 2020-01-22          0       0          0   Jan-2020              0.0   
4 2020-01-22          0       0          0   Jan-2020              0.0   

   Deaths Daily  Recovered Daily  
0           0.0              0.0  
1           0.0              0.0  
2           0.0              0.0  
3           0.0              0.0  
4           0.0              0.0

In [16]:
# Exporting the data

# Setting my path
path = "/Users/datascientistenthusiast/Documents/COVID-19-Analysis"

# Changing my CWD
os.chdir(path)

#full_join.to_csv('CoronaVirus Tableau Raw', sep='\t')
full_join3.to_csv('CoronaVirus Tableau Raw', sep='\t')